In [10]:
import pandas as pd
import requests
import datetime
from datetime import date
# Data from https://optimistic.etherscan.io/

In [11]:
# Contract creator ids
creator_ids = [
    '0xdead1cb30B3CA13Cd67D1d6f4e2790d12484Fdd8',
    '0x03863f6ad36f1fcd908517e3c56c6b3fd3f50752'
]


In [12]:
# Get contracts
df = pd.DataFrame()
apid = []
for addr in creator_ids:
    api_str = 'https://api-optimistic.etherscan.io/api?module=account&action=txlist&address=' + addr + '&startblock=0&endblock=99999999&sort=asc&apikey=YourApiKeyToken'
    print(api_str)
    r = requests.get(api_str)
    print(r)
    x = r.json()
    x = pd.DataFrame(x)
    x = pd.json_normalize(x['result'])
    x = x.loc[(x['to'] == '') & (x['contractAddress'] != '')] #Where transactions are being sent to NULL (means contract creation)
    apid.append(x)

df = pd.concat(apid)
df = df.drop_duplicates() # just in case
contract_hashes = df['contractAddress'].to_numpy() #convert to an array
print(contract_hashes)

https://api-optimistic.etherscan.io/api?module=account&action=txlist&address=0xdead1cb30B3CA13Cd67D1d6f4e2790d12484Fdd8&startblock=0&endblock=99999999&sort=asc&apikey=YourApiKeyToken
<Response [403]>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
#if c

In [ ]:
#Get txs
df = pd.DataFrame()
apid = []
for addr in contract_hashes:
    api_str = 'https://api-optimistic.etherscan.io/api?module=account&action=txlist&address=' + addr + '&startblock=1&endblock=99999999&sort=asc&apikey=YourApiKeyToken'
    r = requests.get(api_str)
    x = r.json()
    x = pd.DataFrame(x)
    x = pd.json_normalize(x['result'])
    x = x.loc[x['to'] == str.lower(addr)] #Where transactions are being sent to the addresses
    #print(x)
    apid.append(x)
df = pd.concat(apid)
#print(df)

In [ ]:
#Aggregate values
df_trunc = pd.DataFrame()
df_trunc['dt'] = pd.to_datetime(df['timeStamp'],unit='s').dt.floor('d') #time is in seconds - trunc to day
df_trunc['gas'] = pd.to_numeric(df['gas'])
df_trunc['gasPrice'] = pd.to_numeric(df['gasPrice'])
df_trunc['gasUsed'] = pd.to_numeric(df['gasUsed'])

print(df_trunc.dtypes)
df_trunc=df_trunc.groupby('dt').agg({'dt':'count','gas':'sum', 'gasPrice':'mean','gasUsed':'sum'})
        .rename(columns={"dt":"num_txs","gas": "total_gas", "gasPrice":"avg_gas_price", "gasUsed":"total_gas_used"})

In [ ]:
#df_export = pd.DataFrame()
#df_export['dt'] = pd.to_datetime(df['timeStamp'],unit='s') #time is in seconds
#df_export['hash'] = df['hash']
#df_export['to'] = df['to']
#df_export['gas'] = df['gas']
#df_export['gasPrice'] = df['gasPrice']
#df_export['gasUsed'] = df['gasUsed']

df_export = df_trunc
df_export = df_export.drop_duplicates() # just in case
export_str = 'optimistic_etherscan_oracle_' + date.today().strftime("%Y_%m_%d") + '.csv'

df_export.to_csv(export_str,index=True)